# Part A: Leaf Node and In-place Operation

In [303]:
import torch

w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

try:
    w += 1.0  
except RuntimeError as e:
    print(e)

output.backward()

print(f"grad: {w.grad}")
print(f"w: {w}")

a leaf Variable that requires grad is being used in an in-place operation.
grad: tensor([3., 2.])
w: tensor([2., 3.], requires_grad=True)


In [304]:
w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

w = w + 1.0

output.backward()

print(w.grad)

None


C:\Users\kingv\AppData\Local\Temp\ipykernel_11428\575195260.py:9: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:494.)
  print(w.grad)


In [305]:
w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

output.backward()
print("Gradient:", w.grad)  

with torch.no_grad():
    w += 1.0          

print("Updated w:", w)

Gradient: tensor([3., 2.])
Updated w: tensor([3., 4.], requires_grad=True)


# Part B: (mini-project) Training

In [306]:
import numpy as np
import pandas as pd

In [307]:
data_df = pd.read_csv("../data/BTCUSDT.csv", sep='|', nrows=10, header=None, usecols=[0,4],
                       names=['timestamp', 'close'])
data_df['timestamp'] = pd.to_datetime(data_df['timestamp'], unit='s')
data_df.set_index('timestamp', inplace=True)
print(data_df)

                       close
timestamp                   
2017-08-17 04:00:00  4261.48
2017-08-17 04:01:00  4261.48
2017-08-17 04:02:00  4280.56
2017-08-17 04:03:00  4261.48
2017-08-17 04:04:00  4261.48
2017-08-17 04:05:00  4261.48
2017-08-17 04:06:00  4261.48
2017-08-17 04:07:00  4261.48
2017-08-17 04:08:00  4261.48
2017-08-17 04:09:00  4261.48


In [308]:
data_tensor = torch.tensor(data_df.values, dtype=torch.float32)
data_tensor, data_tensor.shape

(tensor([[4261.4800],
         [4261.4800],
         [4280.5601],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800]]),
 torch.Size([10, 1]))

In [309]:
w = torch.randn_like(data_tensor, requires_grad=True)

target = torch.tensor(100)

learning_rate = 1e-9

for i in range(20):
    y = w.T @ data_tensor

    loss = (y - target) ** 2

    loss.backward()

    print(f"Epoch: {i}, loss: {loss}")

    with torch.no_grad():
        w -= learning_rate * w.grad

    w.grad.zero_()

print("weights after training : ", w)

Epoch: 0, loss: tensor([[2.2278e+08]], grad_fn=<PowBackward0>)
Epoch: 1, loss: tensor([[90247440.]], grad_fn=<PowBackward0>)
Epoch: 2, loss: tensor([[36558688.]], grad_fn=<PowBackward0>)
Epoch: 3, loss: tensor([[14809699.]], grad_fn=<PowBackward0>)
Epoch: 4, loss: tensor([[5999316.]], grad_fn=<PowBackward0>)
Epoch: 5, loss: tensor([[2430288.5000]], grad_fn=<PowBackward0>)
Epoch: 6, loss: tensor([[984495.6250]], grad_fn=<PowBackward0>)
Epoch: 7, loss: tensor([[398813.5312]], grad_fn=<PowBackward0>)
Epoch: 8, loss: tensor([[161557.4844]], grad_fn=<PowBackward0>)
Epoch: 9, loss: tensor([[65445.5312]], grad_fn=<PowBackward0>)
Epoch: 10, loss: tensor([[26511.8066]], grad_fn=<PowBackward0>)
Epoch: 11, loss: tensor([[10739.8105]], grad_fn=<PowBackward0>)
Epoch: 12, loss: tensor([[4350.6519]], grad_fn=<PowBackward0>)
Epoch: 13, loss: tensor([[1762.4418]], grad_fn=<PowBackward0>)
Epoch: 14, loss: tensor([[713.9960]], grad_fn=<PowBackward0>)
Epoch: 15, loss: tensor([[289.2408]], grad_fn=<PowBack

# Part C: Advanced Indexing and Broadcasting

In [310]:
data_df2 = pd.read_csv("../data/BTCUSDT.csv", sep='|', nrows=100, header=None, usecols=[1, 2, 3, 4, 5],
                       names=['open', 'high', 'low', 'close', 'volume'])
data_tensor2 = torch.tensor(data_df2.values, dtype=torch.float32)
print(data_tensor2[:5])

tensor([[4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 1.7752e+00],
        [4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 0.0000e+00],
        [4.2806e+03, 4.2806e+03, 4.2806e+03, 4.2806e+03, 2.6107e-01],
        [4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 1.2008e-02],
        [4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 1.4080e-01]])


In [311]:
high = data_tensor2[:, 1]
low = data_tensor2[:, 2]
close = data_tensor2[:, 3]

typical_price = (high + low + close) / 3
typical_price.shape

torch.Size([100])

In [312]:
volume = data_tensor2[:, -1]
min_vol = torch.min(volume)
vol_normalized = (volume - min_vol) / (torch.max(volume) - min_vol)
vol_normalized.shape

torch.Size([100])

In [313]:
bullish = data_tensor2[data_tensor2[:, 3] > data_tensor2[:, 0]]
bullish.shape

torch.Size([8, 5])

# Part D: Custom Modules

In [314]:
import torch.nn as nn


class LinearRewardNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1, 5))
        self.register_buffer('profit_target', torch.tensor([[100.0]]))

    def forward(self, x):
        y = x @ self.weights.T 
        return y
    

In [315]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [316]:
data_tensor2 = data_tensor2.to(device)
model = LinearRewardNet().to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8)

epochs = 20

for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()
    y_pred = model(data_tensor2)
    loss = criterion(y_pred, model.profit_target)
    print(f"epoch: {epoch}, loss: {loss}")
    loss.backward()
    optimizer.step()
    

epoch: 0, loss: 30717424.0
epoch: 1, loss: 6992160.0
epoch: 2, loss: 1591631.75
epoch: 3, loss: 362323.0
epoch: 4, loss: 82498.171875
epoch: 5, loss: 18802.2265625
epoch: 6, loss: 4303.37451171875
epoch: 7, loss: 1002.9652099609375
epoch: 8, loss: 251.72073364257812
epoch: 9, loss: 80.70658874511719
epoch: 10, loss: 41.77900314331055
epoch: 11, loss: 32.914188385009766
epoch: 12, loss: 30.89735221862793
epoch: 13, loss: 30.437728881835938
epoch: 14, loss: 30.3337345123291
epoch: 15, loss: 30.309885025024414
epoch: 16, loss: 30.304393768310547
epoch: 17, loss: 30.303050994873047
epoch: 18, loss: 30.302913665771484
epoch: 19, loss: 30.30229949951172


c:\Users\kingv\anaconda3\envs\pytorch_env\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [317]:
model.eval()
with torch.no_grad():
    predict = model(data_tensor2)

    print(predict[:5])

tensor([[101.2942],
        [ 99.0562],
        [ 99.8284],
        [ 99.0713],
        [ 99.2336]], device='cuda:0')
